# Google Maps

In [2]:
#Importamos las librerias que usaremos
import pandas as pd
import os
from datetime import datetime
import dask.dataframe as dd
import ast

C:\Users\LENOVO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\dask\dataframe\__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


## Metadata sitios
A continuacion se realizara la union de los distintos data sets de los datos de negocios y se filtraran por la categoria de restaurantes

In [2]:
ruta_entrada = '../Data/Data_limpia/Google/Metadata'
ruta_salida = '../Data/Data_transformada/Metadata'

# Creamos una lista vacia que va a almacenar todos los data frames 
dfs = []

        # Iteramos la carpeta para leer los archivos
for item in os.listdir(ruta_entrada):
     # Verificamos que sea un archivo JSON y guardamos su ruta
    if item.endswith('.json'):
        path_item = os.path.join(ruta_entrada,item)

        # Leemos el archivo JSON
        df = pd.read_json(path_item, lines = True)

        # Añadimos el DF a la lista de DFS
        dfs.append(df)

# Concatenamos los dataframes para tener un archivo unico      
df_combinado = pd.concat(dfs) 

# Definimos la ruta de salida
salida = os.path.join(ruta_salida, 'Sitios_combinada.parquet')

# Guardamos el data frame en formato parquet
df_combinado.to_parquet(salida, index = False)

In [3]:
# Leemos el archivo de forma porcionada ya que es un data set muy grande
df = dd.read_parquet('../Data/Data_transformada/Metadata/Sitios_combinada.parquet')

In [4]:
df.head(5)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,<NA>,32.388300,-83.357100,[Pharmacy],4.9,16,<NA>,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,<NA>,34.018891,-118.215290,[Textile exporter],4.5,6,<NA>,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,<NA>,34.058092,-118.292130,[Korean restaurant],4.4,18,<NA>,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
3,Nova Fabrics,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",0x80c2c89923b27a41:0x32041559418d447,<NA>,34.023669,-118.232930,[Fabric store],3.3,6,<NA>,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Accessibility': None, 'Activities': None, 'A...",Open ⋅ Closes 5PM,"[0x80c2c8811477253f:0x23a8a492df1918f7, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
4,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,<NA>,34.036694,-118.249421,[Fabric store],4.3,7,<NA>,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Accessibility': None, 'Activities': None, 'A...",Open ⋅ Closes 5PM,"[0x80c2c62c496083d1:0xdefa11317fe870a1, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


In [5]:
# Tomamos las columnas que nos seran utiles
df_2 = df[['name','gmap_id','latitude','longitude','category','avg_rating','num_of_reviews','address']]

In [64]:
# Lo transformamos a un Data frame de Pandas
df_3 = pd.DataFrame(df_2)

In [66]:
# Renombramos las columnas
df_3.rename(columns={0:'name', 1:'gmap_id', 2:'latitude', 3:'longitude', 4:'category', 5:'avg_rating', 6:'num_of_reviews', 7:'address'}, inplace=True)

In [67]:
# Función para convertir la cadena en una lista
def str_to_list(s):
    if pd.isna(s):
        return []  # Manejar valores nulos devolviendo una lista vacía
    # Reemplaza los saltos de línea y espacios después de comillas simples con comas
    s = s.replace('\n', '').replace("' '", "','")

    # Convierte la cadena a una lista de Python
    return ast.literal_eval(s)

In [68]:
# Aplicamos la funcion a la columna 'category'
df_3['category'] = df_3['category'].apply(str_to_list)

In [72]:
# Extraemos todas las categorias por restaurante
df_4 = df_3.explode('category').reset_index(drop=True)

In [73]:
# Verificamos que este de forma correcta
df_4

,name,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,address
0,Porter Pharmacy,0x88f16e41928ff687:0x883dad4fd048e8f8,32.3883,-83.3571,Pharmacy,4.9,16,"Porter Pharmacy, 129 N Second St, Cochran, GA ..."
1,City Textile,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,34.018891,-118.21529,Textile exporter,4.5,6,"City Textile, 3001 E Pico Blvd, Los Angeles, C..."
2,San Soo Dang,0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.29213,Korean restaurant,4.4,18,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ..."
3,Nova Fabrics,0x80c2c89923b27a41:0x32041559418d447,34.023669,-118.23293,Fabric store,3.3,6,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ..."
4,Nobel Textile Co,0x80c2c632f933b073:0xc31785961fe826a6,34.036694,-118.249421,Fabric store,4.3,7,"Nobel Textile Co, 719 E 9th St, Los Angeles, C..."
...,...,...,...,...,...,...,...,...
6202452,Profiles Hair Design,0x8758dd1e2533c991:0x5f6d112918f07dba,43.029664,-108.380849,Beauty salon,4.7,76,"Profiles Hair Design, 522 N Federal Blvd, Rive..."
6202453,Profiles Hair Design,0x8758dd1e2533c991:0x5f6d112918f07dba,43.029664,-108.380849,Hair salon,4.7,76,"Profiles Hair Design, 522 N Federal Blvd, Rive..."
6202454,Arizona LINE-X,0x872b6f6f24aea445:0x22df9e5dd01e67ba,33.680066,-112.113367,Truck accessories store,4.6,24,"Arizona LINE-X, 21242 N Black Canyon Hwy #A, P..."
6202455,Arizona LINE-X,0x872b6f6f24aea445:0x22df9e5dd01e67ba,33.680066,-112.113367,Powder coating service,4.6,24,"Arizona LINE-X, 21242 N Black Canyon Hwy #A, P..."


In [77]:
# Filtramos por el rubro que escogimos el cual fue restaurantes
df_filtrado = df_4[df_4["category"] == "Restaurant"]

In [80]:
df_filtrado.reset_index(drop=True, inplace=True)

In [94]:
# Eliminamos los valores nulos
df_filtrado.dropna(inplace=True)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_33596\3685594660.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado.dropna(inplace=True)


In [95]:
# Verificamos los tipos de datos por columna para ver si hay que cambiar algun tipo de dato
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97026 entries, 0 to 97238
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            97026 non-null  object
 1   gmap_id         97026 non-null  object
 2   latitude        97026 non-null  object
 3   longitude       97026 non-null  object
 4   category        97026 non-null  object
 5   avg_rating      97026 non-null  object
 6   num_of_reviews  97026 non-null  object
 7   address         97026 non-null  object
dtypes: object(8)
memory usage: 6.7+ MB


In [101]:
# Creamos una funcion para extraer la ciudad desde la direccion
def extraer_ciudad(direccion):

    try:
        # Dividir la dirección en partes usando la coma como separador
        partes = direccion.split(", ")

        # Asegurarse de que haya al menos tres partes (ciudad, estado_código postal)
        if len(partes) >= 3:
            ciudad = partes[2]  # Extraer la 3ra parte como ciudad
            return ciudad
        else:
            return None # Devolver None para ciudad y estado si tiene la configuración planteada
    except IndexError:
        return None  # Devolver None para ciudad y estado si la extracción falla

In [103]:
# Agregamos una columna para colocar la ciudad
df_filtrado["city"] = df_filtrado['address'].apply(extraer_ciudad)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_33596\1119377874.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado["city"] = df_filtrado['address'].apply(extraer_ciudad)


In [105]:
# Eliminamos la columna de direccion, ya que la ubicacion exacta la obtenemos con la latitud y longitud
df_filtrado.drop(columns='address', inplace=True)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_33596\2184024267.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado.drop(columns='address', inplace=True)


In [110]:
# Cambiamos el tipo de dato de las columnas que lo requieren para su correcto funcionamiento
df_filtrado['latitude'] = df_filtrado['latitude'].astype(float)
df_filtrado['longitude'] = df_filtrado['longitude'].astype(float)
df_filtrado['avg_rating'] = df_filtrado['avg_rating'].astype(float)
df_filtrado['num_of_reviews'] = df_filtrado['num_of_reviews'].astype(int)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_33596\1480663786.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['latitude'] = df_filtrado['latitude'].astype(float)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_33596\1480663786.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['longitude'] = df_filtrado['longitude'].astype(float)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_33596\1480663786.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [111]:
# Realizamos una ultima validacion para verificar que los tipos de datos sean correctos y que no hayan valores nulos
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97026 entries, 0 to 97238
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            97026 non-null  object 
 1   gmap_id         97026 non-null  object 
 2   latitude        97026 non-null  float64
 3   longitude       97026 non-null  float64
 4   category        97026 non-null  object 
 5   avg_rating      97026 non-null  float64
 6   num_of_reviews  97026 non-null  int32  
 7   city            97026 non-null  object 
dtypes: float64(3), int32(1), object(4)
memory usage: 6.3+ MB


In [112]:
# Exportamos el dataset en formato parquet
df_filtrado.to_parquet('../Data/Parquet/Sitios.parquet', index=False)

## Reviews por estado
A continuacion se realizara la union de los distintos datasets que se nos brindaron para tener en un solo archivo los datos de los reviews y filtrado por la categoria de negocio elegida ('Restaurant')

In [2]:
# Guardamos como variables los directorios de los archivos que vamos a leer y a donde los queremos guardar
ruta_entrada = '../Data/Data_limpia/Google/Reviews'
ruta_salida = '../Data/Data_transformada/Estados'

# Iteramos el directorio para poder abrir cada carpeta de cada estado

for carpeta in os.listdir(ruta_entrada):
    # guardamos la ruta de la carpeta
    path_carpeta = os.path.join(ruta_entrada,carpeta)

    # Verificamos que sea una carpeta
    if os.path.isdir(path_carpeta):

        # Creamos una lista vacia que va a almacenar todos los data frames 
        dfs = []

        # Iteramos la carpeta para leer los archivos
        for item in os.listdir(path_carpeta):
            # Verificamos que sea un archivo JSON y guardamos su ruta
            if item.endswith('.json'):
                path_item = os.path.join(path_carpeta,item)

                # Leemos el archivo JSON
                df = pd.read_json(path_item, lines = True)

                # Añadimos el DF a la lista de DFS
                dfs.append(df)

        # Concatenamos los dataframes para tener un archivo unico      
        df_combinado = pd.concat(dfs) 

        # Definimos la ruta de salida
        salida = os.path.join(ruta_salida, f'{carpeta}.parquet')

        # Guardamos el data frame en formato parquet
        df_combinado.to_parquet(salida, index = False)

In [3]:
# Leemos uno de los datasets extraidos para verificar que este de la forma correcta
df = pd.read_parquet('../Data/Data_transformada/Estados/review-Alabama.parquet')
df.head(5)

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.140438e+20,Kanisha Mixon,1597168272670,5,Very Personable staff! Beautiful and clean env...,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
1,1.160090e+20,Brandie Hodges,1609899039594,5,Best clothing intown,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
2,1.062399e+20,Sharon King,1547235290843,4,None,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
3,1.049701e+20,Veronica Pierce,1517709403534,5,None,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
4,1.105875e+20,Whitney Waldon Collier,1535245718492,5,None,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf


In [16]:
ruta_entrada = '../Data/Data_transformada/Estados'
ruta_salida = '../Data/Data_transformada/Filtrados'

# Ruta que contiene los sitios
sitios_metadata = "../Data/Parquet/Sitios.parquet"

df_sitios = pd.read_parquet(sitios_metadata)

# Iteramos el directorio donde se encuentran los archivos antes extraidos
for item in os.listdir(ruta_entrada):

    # Cverificamos que sea un archivo parquet
    if item.endswith(".parquet"):
        # Leemos el archivo con Pandas
        df = pd.read_parquet(os.path.join(ruta_entrada, item))
        
        # Funcion para transformar en 'yes' si tiene datos la celda o en 'no' si esta vacia 
        def replace_none(value):
            if value is None:
                return "No"
            else:
                return "Yes"
            
        # Aplicamos la funcion en la columna 'resp' para saber si tiene respuesta el review o no
        df["resp"] = df["resp"].apply(replace_none)
        
        # Eliminamos la columna 'pics' ya que no nos proporciona informacion valiosa
        df.drop(["pics"], axis=1, inplace=True)
        
        # Eliminamos los duplicados
        df.drop_duplicates(inplace=True)

        # Renombramos la columna 'name' la cual contiene el nombre del usuario
        df.rename(columns={"name": "user_name"}, inplace = True)

        # Aplicamos un Merge con el dataset de los sitios para hacer el filtro de los restaurantes
        df = df.merge(
            df_sitios[["gmap_id", "name", "category", "city"]],
            on="gmap_id",
            how="left",
        )

        # Renombramos la columna 'name' la cual pertenece al nombre de la tienda
        df.rename(columns={"name": 'business_name'}, inplace = True)

        # Eliminamos valores nulos en las columnas importantes
        df = df.dropna(subset=["business_name", "city"])

        # Estraemos el estado del nombre del archivo parquet
        state_name = item.split("-")[-1].split(".")[0]

        # Agregamos una columna que especifica el nombre del estado en el que se realizo el review
        df["State_review"] = state_name

        # Configuramos el directorio de salida para cada archivo y se exporta en tipo parquet
        output_file_path = os.path.join(ruta_salida, item)
        df.to_parquet(output_file_path, index=False)


Despues de tener los archivos listos por estado, hacemos un pequeño analisis para ver que estado tendremos en cuenta segun la cantidad de reviews de restaurantes

In [18]:
# Ruta de los archivos ya filtrados
ruta_entrada = '../Data/Data_transformada/Filtrados'

# Creamos una lista vacia para almacenar los dataframes
dfs = []

# Iteramos la carpeta
for item in os.listdir(ruta_entrada):
    # Verificamos que sea un archivo parquet
    if item.endswith(".parquet"):
        # RLeemos el parquet por medio de Pandas
        df = pd.read_parquet(os.path.join(ruta_entrada, item))

        # Agregamos el Dataframe a la lista
        dfs.append(df)

# Concatenamos todos los datos en un solo dataframe para poder compararlos
df_Google_reviews_CS = pd.concat(dfs, ignore_index=True)

In [20]:
# Vemos la cantidad de registros
df_Google_reviews_CS.shape

(5416877, 11)

In [21]:
# Contamos la cantidad de registros por estado
df_Google_reviews_CS['State_review'].value_counts()

State_review
Florida                 498783
California              346806
New_York                292376
Texas                   255458
Georgia                 243832
Michigan                231515
Illinois                227805
North_Carolina          214483
Ohio                    206386
Pennsylvania            173676
Virginia                171090
Arizona                 159897
Washington              148545
Tennessee               139052
New_Jersey              137142
Colorado                136458
Missouri                132160
South_Carolina          121503
Indiana                 121271
Maryland                110951
Wisconsin               105681
Minnesota                93550
Utah                     92652
Alabama                  91798
Oregon                   91640
Massachusetts            87484
Louisiana                77855
Oklahoma                 77748
Kentucky                 66953
Iowa                     58225
Nevada                   48014
Arkansas                 4

Despues de este pequeño analisis decidimos escoger el estado de Florida como enfoque, por lo tanto, realizaremos los ultimos detalles para por ultimo, exportar el archivo.

In [37]:
# Leemos el archivo correspondiente al estado de Florida
florida_df = pd.read_parquet('../Data/Data_transformada/Filtrados/review-Florida.parquet')

In [38]:
# Revisamos los tipos de datos y los valores nulos
florida_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498783 entries, 0 to 498782
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   user_id        498783 non-null  float64
 1   user_name      498783 non-null  object 
 2   time           498783 non-null  int64  
 3   rating         498783 non-null  int64  
 4   text           290721 non-null  object 
 5   resp           498783 non-null  object 
 6   gmap_id        498783 non-null  object 
 7   business_name  498783 non-null  object 
 8   category       498783 non-null  object 
 9   city           498783 non-null  object 
 10  State_review   498783 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 41.9+ MB


In [39]:
# Creamos una funcion que permite extraer la fecha y la hora desde la columna de time

def convert_timestamp_ms_to_datetime(timestamp_ms):
    """
    Convierte un timestamp en milisegundos a un objeto datetime.
    """
    if pd.isna(timestamp_ms):
        return None
    # Convertir a segundos
    timestamp_s = timestamp_ms / 1000.0
    # Convertir a datetime
    return datetime.fromtimestamp(timestamp_s)

In [40]:
# Aplicamos la funcion a la columna de time
florida_df['time'] = florida_df['time'].apply(convert_timestamp_ms_to_datetime)

In [41]:
# Creamos las columnas de fecha, hora y dia de la semana a partir de la fecha extraida anteriormente
florida_df['date'] = florida_df['time'].dt.normalize()
florida_df['hour'] = florida_df['time'].dt.hour
florida_df['day'] = florida_df['time'].dt.day_name()

In [32]:
# Verificamos que este correcto
florida_df.head(5)

,user_id,user_name,time,rating,text,resp,gmap_id,business_name,category,city,State_review,date,hour,day
0,1.130683e+20,Edwin Ashurst,2018-09-17 09:51:08.022,5,The salads were lackluster though with well wr...,No,0x8894b5a7a7909725:0xe4687299fd288188,Gormley's on the river | Modern cuisine in his...,Restaurant,Apalachicola,Florida,2018-09-17,9,Monday
1,1.130683e+20,Edwin Ashurst,2018-09-17 09:51:08.022,5,The salads were lackluster though with well wr...,No,0x8894b5a7a7909725:0xe4687299fd288188,Gormley's on the river | Modern cuisine in his...,Restaurant,Apalachicola,Florida,2018-09-17,9,Monday
2,1.059416e+20,Douglas Gummere,2018-09-19 19:12:47.631,1,"Love this place. Great little restaurant, with...",No,0x8894b5a7a7909725:0xe4687299fd288188,Gormley's on the river | Modern cuisine in his...,Restaurant,Apalachicola,Florida,2018-09-19,19,Wednesday
3,1.059416e+20,Douglas Gummere,2018-09-19 19:12:47.631,1,"Love this place. Great little restaurant, with...",No,0x8894b5a7a7909725:0xe4687299fd288188,Gormley's on the river | Modern cuisine in his...,Restaurant,Apalachicola,Florida,2018-09-19,19,Wednesday
4,1.098577e+20,David Powell,2018-08-22 06:13:16.392,5,A classy restaurant to match the charm of the ...,No,0x8894b5a7a7909725:0xe4687299fd288188,Gormley's on the river | Modern cuisine in his...,Restaurant,Apalachicola,Florida,2018-08-22,6,Wednesday


In [43]:
# Eliminamos la columna time, ya que los datos los extraimos en otras columnas
florida_df.drop(columns={'time'}, inplace = True)

In [45]:
# Eliminamos los valores nulos ya que el dataset aun es muy grande y necesitamos reducir el volumen de datos
florida_df.dropna(inplace=True)
florida_df.reset_index(inplace=True)

In [46]:
# Validar que los tipos de datos esten correctos
florida_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290721 entries, 0 to 290720
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   index          290721 non-null  int64         
 1   user_id        290721 non-null  float64       
 2   user_name      290721 non-null  object        
 3   rating         290721 non-null  int64         
 4   text           290721 non-null  object        
 5   resp           290721 non-null  object        
 6   gmap_id        290721 non-null  object        
 7   business_name  290721 non-null  object        
 8   category       290721 non-null  object        
 9   city           290721 non-null  object        
 10  State_review   290721 non-null  object        
 11  date           290721 non-null  datetime64[ns]
 12  hour           290721 non-null  int32         
 13  day            290721 non-null  object        
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)

In [47]:
# Reducimos de forma aleatoria el dataset a 200000 valores
florida_reducido = florida_df.sample(n=200000, random_state=7)

In [50]:
# Vemos el rango de fechas que tenemos para el analisis

fecha_minima = florida_reducido['date'].min()
fecha_maxima = florida_reducido['date'].max()

print(f'EL rango de fechas para el analisis va desde el {fecha_minima} hasta el {fecha_maxima}')

Timestamp('2021-09-07 00:00:00')

In [48]:
# Exportamos el dataset en un archivo tipo parquet
florida_reducido.to_parquet('../Data/Parquet/Reviews_florida.parquet', index=False)

In [3]:
import pandas as pd

sitios = pd.read_parquet('../Data/Parquet/business.parquet')
sitios.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,W57cR9a7XP6RX56MSWHJJA,Las Margaritas,514 S Rampart St,New Orleans,FL,70163,29.950290,-90.074887,3.5,58,0,"{'AgesAllowed': None, 'Alcohol': 'u'full_bar''...","Mexican, Restaurants","{'Friday': '11:0-22:30', 'Monday': '0:0-0:0', ..."
1,xa6JYHDgVza7CuenKBJBHw,Taqueria El Sitio,3466 A Lebanon Pike,Hermitage,FL,37076,36.191654,-86.625419,4.5,8,0,"{'AgesAllowed': None, 'Alcohol': None, 'Ambien...","Latin American, Mexican, Restaurants","{'Friday': '10:0-21:0', 'Monday': None, 'Satur..."
2,XwoXi0M66RU6Gouq-DSDHw,G Peppers Grill & Tavern,8745 Gunn Hwy,Odessa,FL,33556,28.092136,-82.578574,4.0,82,1,"{'AgesAllowed': None, 'Alcohol': 'u'beer_and_w...","Restaurants, Mexican, American (Traditional)","{'Friday': '11:0-23:0', 'Monday': '11:0-22:0',..."
3,OfW-VK04qr7b7bPm5QbyLg,Taqueria Feliz,303 Horsham Rd,Horsham,FL,19044,40.180727,-75.134699,3.0,77,0,"{'AgesAllowed': None, 'Alcohol': 'u'full_bar''...","Mexican, Pizza, Restaurants","{'Friday': '11:30-22:0', 'Monday': '11:30-21:0..."
4,YhQAQhG1IkXJH7oLZMbM9w,Del Taco,252 Los Altos Pkwy,Sparks,FL,89431,39.583653,-119.740935,1.5,60,1,"{'AgesAllowed': None, 'Alcohol': ''none'', 'Am...","Restaurants, Fast Food, Mexican","{'Friday': '8:0-12:0', 'Monday': '8:0-23:0', '..."


In [4]:
sitios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 867 entries, 0 to 866
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   867 non-null    object 
 1   name          867 non-null    object 
 2   address       867 non-null    object 
 3   city          867 non-null    object 
 4   state         867 non-null    object 
 5   postal_code   867 non-null    object 
 6   latitude      867 non-null    float64
 7   longitude     867 non-null    float64
 8   stars         867 non-null    float64
 9   review_count  867 non-null    int64  
 10  is_open       867 non-null    int64  
 11  attributes    855 non-null    object 
 12  categories    867 non-null    object 
 13  hours         745 non-null    object 
dtypes: float64(3), int64(2), object(9)
memory usage: 95.0+ KB
